In [ ]:
# -------------------------------
# 0. 라이브러리 및 환경 설정
# -------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from scipy.stats import zscore, pointbiserialr
from sklearn.preprocessing import PowerTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 그래프 한글폰트(선택) ─ macOS 예시
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# -------------------------------
# 1. 데이터 불러오기
# -------------------------------
### TODO: 실제 파일 경로를 지정하세요
df = pd.read_parquet("../01_fetch/financial.parquet")

# -------------------------------
# 2. 주요 변수 지정
# -------------------------------
### TODO: 컬럼명을 실제 데이터에 맞춰 수정
id_col      = 'ticker'
target_col  = 'label'          # 부실=1, 정상=0
num_cols    = [                # 수치형 재무/파생 변수 목록
    'total_assets', 'total_liab',
    'sales', 'net_income',
    'current_ratio', 'debt_ratio',
    'altman_k2',  # …
]
cat_cols = []                  # 범주형 변수 있다면 추가

In [ ]:
# 2-1. 정보 요약
display(df.info())
display(df.head())

# 2-2. 중복 확인 (동일 기업·연도 중복행 탐지)
dup_mask = df.duplicated(subset=[id_col, 'year'])
print(f"중복 행 수: {dup_mask.sum()}")
if dup_mask.any():
    df = df[~dup_mask]  # 필요 시 제거

In [ ]:
# 3-1. 변수별 결측 비율
null_ratio = df[num_cols + [target_col]].isna().mean().sort_values(ascending=False)
display(null_ratio.head(10))

# 3-2. 히트맵 시각화
plt.figure(figsize=(10,4))
msno.matrix(df[num_cols + [target_col]].sample(min(500, len(df))))
plt.show()

In [ ]:
# 4-1. 기술통계
desc = df[num_cols].describe().T
display(desc)

# 4-2. z-score 기반 이상치 마스크
z_scores = df[num_cols].apply(zscore)
outlier_mask = (np.abs(z_scores) > 3).any(axis=1)
print(f"z>|3| 이상치 행 비율: {outlier_mask.mean():.2%}")

# 4-3. 상자그림(샘플 6개 변수)
plot_cols = num_cols[:6]
sns.boxplot(data=df[plot_cols], orient='h'); plt.title("Boxplot (sample)"); plt.show()

In [ ]:
# 5-1. 오른쪽 꼬리가 긴 양수 변수에 log1p 적용 예시
log_cols = ['sales', 'total_assets']    # TODO: 필요 변수 지정
df[log_cols] = df[log_cols].apply(np.log1p)

# 5-2. Yeo-Johnson 변환 예시
yj_cols = ['current_ratio']             # TODO: 필요 변수 지정
pt = PowerTransformer(method='yeo-johnson')
df[yj_cols] = pt.fit_transform(df[yj_cols])

In [ ]:
# 6-1. 피어슨 상관행렬
corr = df[num_cols].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
plt.title("Correlation Matrix"); plt.show()

# 6-2. VIF 계산
X = df[num_cols].drop(columns=[], errors='ignore')  # 필요 시 제외 변수 지정
vif_vals = pd.Series(
    [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    index=X.columns, name='VIF'
)
display(vif_vals.sort_values(ascending=False).head(10))

In [ ]:
# 7-1. 포인트-바이시리얼 상관계수
pb_dict = {
    col: pointbiserialr(df[col].fillna(0), df[target_col])[0]
    for col in num_cols
}
pb_series = pd.Series(pb_dict, name='point_biserial')
display(pb_series.sort_values(key=np.abs, ascending=False).head(10))

# 7-2. 변수별 상자그림 (부실 vs 정상)
plt.figure(figsize=(12,4))
for i, c in enumerate(num_cols[:3], 1):    # 샘플 3개 변수
    plt.subplot(1,3,i)
    sns.boxplot(x=target_col, y=c, data=df)
    plt.title(c)
plt.tight_layout(); plt.show()

In [ ]:
class_counts = df[target_col].value_counts().sort_index()
print(class_counts)
print("부실(1) 비율:", class_counts.get(1,0) / class_counts.sum())

# 파이차트
plt.figure(figsize=(4,4))
class_counts.plot.pie(autopct='%1.1f%%', labels=['정상(0)','부실(1)'])
plt.ylabel('')
plt.title("Class Distribution"); plt.show()